# Video Thumbnails

In this example we will create a `Table` with `Url`s to videos, where we create a thumbnail for each video.

We use a subset of the [UCF11 YouTube Action dataset](https://www.crcv.ucf.edu/data/UCF_YouTube_Action.php), which consists of short clips, each with an associated category. It is structured as follows:

```
UCF11/
├─ basketball/
│  ├─ v_shooting_01
|  │  ├─ v_shooting_01_01.mpg
|  │  ├─ v_shooting_01_02.mpg
|  │  ├─ ...
│  ├─ v_shooting_02
|  │  ├─ ...
├─ biking/
│  ├─ ...
├─ ...
```

The following code recurses over the video files, creating a thumbnail image and a row for each. These are written to
a 3LC `Table` using a `TableWriter`.

## Imports

In [ ]:
from pathlib import Path

import cv2
import tlc

## Project setup

In [ ]:
DATASET_LOCATION = Path("../../data/ucf11").absolute()

assert DATASET_LOCATION.exists(), f"Dataset not found at {DATASET_LOCATION}"

## Create Table

The class names are read from the directory names.

In [ ]:
class_directories = [path for path in DATASET_LOCATION.glob("*") if path.is_dir()]

In [ ]:
classes = [c.name for c in class_directories]
classes

We now define a schema for the `Table`. Each row should consist of a generic `Url` to the video file, the thumbnail image and a categorical label for the video class.

In [ ]:
column_schemas = {
    "video_url": tlc.Path("video_url"),
    "thumbnail": tlc.ImagePath("thumbnail"),
    "label": tlc.CategoricalLabel("label", classes=classes),
}

We then iterate over the videos, write the thumbnails next to the videos, and write the `Table` with a `TableWriter`.

In [ ]:
def create_thumbnail(video_path: Path, thumbnail_path: Path) -> None:
    """Read the first frame of a video and save it to disk.

    :param video_path: Path to the video file.
    :param thumbnail_path: Path to save the thumbnail image.
    """
    cap = cv2.VideoCapture(str(video_path))
    _, frame = cap.read()
    cv2.imwrite(str(thumbnail_path), frame)
    cap.release()

In [ ]:
table_writer = tlc.TableWriter(
    project_name="3LC Tutorials",
    dataset_name="UCF YouTube Actions",
    table_name="initial",
    column_schemas=column_schemas,
)

for class_idx, class_directory in enumerate(class_directories):
    for video_path in class_directory.rglob("*mpg"):
        video_path = video_path.absolute()
        # Create a thumbnail
        thumbnail_path = video_path.with_suffix(".jpg")
        create_thumbnail(video_path, thumbnail_path)

        # Write the row
        row = {
            "video_url": str(video_path),
            "thumbnail": str(thumbnail_path),
            "label": class_idx,
        }

        table_writer.add_row(row)

table = table_writer.finalize()

In [ ]:
table